In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import seaborn as sns
import requests as req
import sys
import time
sys.path.insert(0, '../../../API_keys/')
from vantage_api import premium_key
import json

This file is too large to upload entirely to GitHub and most inputs are commented out. To run, upload the filled_reps_Data.csv from https://drive.google.com/drive/folders/1LG5bKuLBJXHF2HL9jAuEtyCQbmk8m3lS to the datasets folder. Then, run each line of code. We have the images of our EDA already saved and can be seen in the pics folder.

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
#uncomment this line
#df = pd.read_csv('datasets/filled_reps_Data.csv') #Too large to push to GitHub if run

In [ ]:
df.head()

In [ ]:
df.drop(columns='Unnamed: 0',inplace=True)
df.columns

The '7/15/2022' is at the last column in the dataframe and is the value of the trade

In [ ]:
val_dict = {}
for i in df.name.unique():
    val = 0
    for j in df.index:
        if df['name'][j] == i:
            val += df['7/15/2022'][j]
    val_dict.update({i:val})
df['values'] = df['name'].map(lambda x: val_dict[x])
df.head()

Now that we have the final values, let's do some EDA

In [ ]:
millions_list = [n for n in val_dict if val_dict[n] >= 1000000]
millions_list

We have 2 Senators/Representatives that made over a million dollars in trades. Let's examin them.

In [ ]:
look_at = ['name', 'chamber', 'party', 'birthday', 'gender', 'represents', 'values']
df[df['name'] == 'Michael T. McCaul'][look_at].head(1)

In [ ]:
df[df['name'] == 'Greg Gianforte'][look_at].head(1)

Let's look at the data as a whole to see if there are any indicators to see who benifits in Congress.

In [ ]:
sns.boxplot(data=df, x='party', y='values')

This shows that the Republicans tend to make more in stock trades than the Democrats

In [ ]:
sns.boxplot(data=df, x='gender', y='values')

Males in Congress seem to make more than females.

In [ ]:
sns.boxplot(data=df, x='chamber', y='values')

Senators seem to make more than the House members

In [ ]:
df.head()

Let's make the columns more usable. 

In [ ]:
df['senate'] = df['chamber'].map(lambda x: 1 if x == 'senate' else 0)
df['republican'] = df['party'].map(lambda x: 1 if x == 'Republican' else 0)
df['male'] = df['gender'].map(lambda x: 1 if x == 'M' else 0)
df['age'] = df['birthday'].map(lambda x: 2022 - int(x[-4::]))

In [ ]:
df_heat = df[['senate', 'republican', 'male', 'age', 'values']]

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df_heat.corr()[['values']].sort_values(by='values',ascending=False),
            annot = True,
            vmin = -1,
            vmax = 1,
            cmap = 'coolwarm');

It seems like the best predictor of the returns rate is whether or not the trader was male. Males make more than females in the Congress. However, it isn't too strongly correlated.

In [ ]:
sns.scatterplot(data=df_heat,x='age',y='values')

This shows the distrabution of earnings by age.

Finally, let's see what percent actually make money on the market, as opposed to loosing

In [ ]:
names_len = len(df['name'].unique())
names_len

In [ ]:
ratio = 0
for i in df['name'].unique():
    if val_dict[i] > 0:
        ratio += 1
ratio /= names_len
ratio

Only 27.72% of Congress humans actually make a profit on the stock market.